In [1]:
import torch
from diffusers import AnimateDiffPipeline, LCMScheduler, MotionAdapter
from diffusers.utils import export_to_gif

/Users/captain2mac/Documents/Code/HuggingFace/.venv/lib/python3.9/site-packages/diffusers/utils/outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
adapter = MotionAdapter.from_pretrained("wangfuyun/AnimateLCM", torch_dtype=torch.float16)
pipe = AnimateDiffPipeline.from_pretrained("emilianJR/epiCRealism", motion_adapter=adapter, torch_dtype=torch.float16)
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config, beta_schedule="linear")

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

/Users/captain2mac/Documents/Code/HuggingFace/.venv/lib/python3.9/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(
The config attributes {'center_input_sample': False, 'flip_sin_to_cos': True, 'freq_shift': 0, 'mid_block_type': 'UNetMidBlock2DCrossAttn', 'only_cross_attention': False, 'attention_head_dim': 8, 'dual_cross_attention': False, 'class_embed_type': None, 'addition_embed_type': None, 'num_class_embeds': None, 'upcast_attention': False, 'resnet_time_scale_shift': 'default', 'resnet_skip_time_act': False, 'resnet_out_scale_factor': 1.0, 'time_embedding_type': 'positional', 'time_embedding_dim': None, 'time_embedding_act_fn': None, 'timestep_post_act': None, 'time_cond_proj_dim': None, 'conv_in_kernel': 3, 'conv_out_kernel': 3, 'projection_class_embeddings_input_dim': None, 'class_embeddings_con

In [3]:

pipe.load_lora_weights("wangfuyun/AnimateLCM", weight_name="AnimateLCM_sd15_t2v_lora.safetensors", adapter_name="lcm-lora")
# pipe.set_adapters(["lcm-lora"], [0.8])

# pipe.enable_vae_slicing()
pipe = pipe.to('mps')

In [6]:
output = pipe(
    prompt="UI/UX of pet social media application",
    negative_prompt="bad quality, worse quality, low resolution, watermark, artefact, weird anatomy",
    num_frames=16,
    guidance_scale=2.0,
    num_inference_steps=6,
)

  0%|          | 0/6 [00:00<?, ?it/s]

RuntimeError: MPS backend out of memory (MPS allocated: 21.45 GB, other allocations: 7.74 GB, max allowed: 36.27 GB). Tried to allocate 8.00 GB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [5]:
from datetime import datetime

time = datetime.now().strftime("%hh_%mm_%ss")

frames = output.frames[0]
export_to_gif(frames, f"../outputs/videos/animatelcm_{time}.gif")

'animatelcm_HH_mm_ss.gif'